In [ ]:
# Import libraries
import pandas as pd
import pickle

In [ ]:
# Load the best-trained model
with open('modelo_lgb_tfidf_unigram_ptbr.pkl', 'rb') as file:
    modelo_lgb_tfidf_unigram_ptbr = pickle.load(file)

In [ ]:
# To load the TF-IDF vectorizer from the file using pickle
with open('tfidf_vec_unigram_ptbr.pkl', 'rb') as file:
    tfidf_vec_unigram_ptbr = pickle.load(file)

In [ ]:
tfidf_vec_unigram_ptbr

In [ ]:
# Load File
df_n = pd.read_csv('./dataset_III_pre_process_II.csv')

In [ ]:
df_n

In [ ]:
####
# Column full_text (X_text_n)

X_text_n = df_n['full_text']

In [ ]:
# Use the TF-IDF vectorizer to transform the new texts into TF-IDF vectors - Unigram
X_tfidf_unigram_n = tfidf_vec_unigram_ptbr.transform(X_text_n)

In [ ]:
X_tfidf_unigram_n.shape

In [ ]:
# Use the LightGBM model to make predictions on the new posts
prev = modelo_lgb_tfidf_unigram_ptbr.predict(X_tfidf_unigram_n)

In [ ]:
# Create a new DataFrame to store the predictions
df_previsoes = pd.DataFrame({'full_text_X_tfidf_unigram_n': X_tfidf_unigram_n, 'previsoes': prev})

In [ ]:
df_previsoes

In [ ]:
# Create a new column 'predictions' in the DataFrame 'df_n' with the probabilities
df_n['probabilidade'] = prev

In [ ]:
df_n

In [ ]:
# Probability ranges

# Defining the probability ranges
faixas = [0, 0.3, 0.7, 1.0]
nomes_faixas = ['Baixa', 'Média', 'Alta']

# Adding a new column to the DataFrame that contains the probability categories
df_n['Relevância'] = pd.cut(df_n['probabilidade'], bins=faixas, labels=nomes_faixas, right=False)

In [ ]:
df_n

In [ ]:
# Threshold for binary classification
limiar = 0.5

# Convert the probabilities into binary predictions
previsoes_binarias = [1 if prob >= limiar else 0 for prob in prev]

# Create a new column 'predictions' in the DataFrame 'df_n' with the binary predictions
df_n['previsao_binaria'] = previsoes_binarias

In [ ]:
df_n

In [ ]:
# Save the new posts with the predictions
df_n.to_csv('dataset_III_prob_relevancia_prev_bin.csv', index=False)
df_n.to_excel('dataset_III_prob_relevancia_prev_bin.xlsx', index=False)

In [ ]:
# GRAPH
import matplotlib.pyplot as plt

# Count of categories in the 'Relevance' column
contagem_relevancia = df_n['Relevância'].value_counts()

# Create the bar chart
plt.bar(contagem_relevancia.index, contagem_relevancia.values)

# Add title and labels to the axes
plt.title('Contagem de Relevância')
plt.xlabel('Relevância')
plt.ylabel('Contagem')

# Display the chart
plt.show()

In [ ]:
# PIE CHART WITH PERCENTAGES


# Calculate the percentages of categories in the 'Relevance' column
percentuais_relevancia = df_n['Relevância'].value_counts(normalize=True) * 100

# Calculate the overall total
total_geral = len(df_n)

# Define the colors for each category
cores = ['green', 'yellow', 'red']

# Create the pie chart
plt.figure(figsize=(6, 6))  # Define o tamanho da figura
plt.pie(percentuais_relevancia, labels=percentuais_relevancia.index, autopct='%.1f%%', startangle=140, colors=cores)


# Add title
#plt.title('Percentuais de Relevância dos', total_geral, 'Posts Classificados')
plt.title('Percentuais de Relevância dos ' + str(total_geral) + ' Novos Posts Classificados')


# Create a custom legend
legendas = ['Baixa', 'Média', 'Alta']
plt.legend(legendas, title="Relevância", loc="best")

# Display the chart
plt.show()

In [ ]:
# Print the quantity of each range
contagem_faixas = df_n['Relevância'].value_counts()
print(contagem_faixas)